In [1]:
import papermill as pm
from nbconvert import PDFExporter
import nbformat

from orbit_generation.experiment import generate_parameter_sets

Parameters

In [2]:
params = {
    # Data
    'data_used': 'EM_N_fix_1500',
    'families_to_discard': [0, 2, 4, 10, 20],
    'seq_len': 100,
    'feature_dim': 7,
    
    # Training
    'epochs': 50,
    'val_split': 0.05,
    'batch_size': 32,
    'lr': 0.001,
    
    # Model
    'model_name': ['vae_conv5_legit', 'inception_time_wp_vae'],
    'latent_dim': [2, 6],
    'beta': [0.001, 0.2, 0.5, 1, 1.5, 2, 10],
    
    # Convergence
    'max_iter_convergence': 20,
    'input_seq_len_convergence': 1, ### make experiment first
    
    # Evaluation
    'samples_to_generate': 100,
    'distance_metric': 'euclidean'
}

In [3]:
model_specific_params = {
    'vae_conv5_legit': {
        'dropout_rate': 0.2
    },
    'inception_time_wp_vae': {
        'n_filters': 32,
        'kernel_sizes': [3, 7, 13],
        'bottleneck_channels': 32
    }
}

Parameter Set

In [4]:
parameter_sets = generate_parameter_sets(params, model_specific_params)

In [5]:
len(parameter_sets)

140

In [6]:
import os
import json
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import papermill as pm
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [7]:
def execute_notebook(notebook_to_execute, output_dir, i, **kwargs):
    try:
        # Generate output filenames
        base_name = os.path.splitext(os.path.basename(notebook_to_execute))[0]
        output_notebook = os.path.join(output_dir, f"{base_name}_execution_{i}.ipynb")
        
        # Check if output notebook already exists
        if os.path.exists(output_notebook):
            logging.info(f"Skipping execution {i}, output already exists.")
            return i
        
        # Read the notebook
        with open(notebook_to_execute, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Find the parameters cell and update it
        params_cell_found = False
        for index, cell in enumerate(nb.cells):
            if cell.cell_type == 'code' and cell.source.startswith('# parameters'):
                # Update the cell source with new parameter values
                new_source = "# parameters\n"
                for key, value in kwargs.items():
                    new_source += f"{key} = {repr(value)}\n"
                cell.source = new_source
                params_cell_found = True
                
                # Add a cell to print out and verify the parameters
                verify_params_cell = nbformat.v4.new_code_cell(
                    source="print('Injected parameters:', {" + 
                           ", ".join(f"'{k}': {k}" for k in kwargs.keys()) + 
                           "})"
                )
                nb.cells.insert(index + 1, verify_params_cell)
                break

        if not params_cell_found:
            raise ValueError("Parameters cell not found in the notebook")

        # Execute only the parameters cell and the verification cell
        ep = ExecutePreprocessor(timeout=600, kernel_name='pytorch')
        ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_to_execute)}})

        # Check if parameters were properly injected
        if len(nb.cells) > index + 1 and nb.cells[index + 1].outputs:
            injected_params = nb.cells[index + 1].outputs[0].text
            logging.info(f"Injected parameters for execution {i}: {injected_params}")
            
            # Verify that all parameters are present
            for key in kwargs.keys():
                if key not in injected_params:
                    raise ValueError(f"Parameter '{key}' was not properly injected")
        else:
            raise ValueError("Failed to verify injected parameters")

        # Now execute the rest of the notebook
        nb = pm.execute_notebook(
            nb,
            output_notebook,
            parameters={},
            kernel_name='pytorch',
            timeout=10000
        )
        
        logging.info(f"Completed execution {i}")
        return i

    except Exception as e:
        logging.error(f"Error in execution {i}: {str(e)}")
        logging.error(f"Parameters used: {kwargs}")
        import traceback
        logging.error(f"Traceback: {traceback.format_exc()}")
        return None

In [8]:
import os
import json
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_parameter_sets(parameter_sets, notebook_to_execute, output_dir, checkpoint_file, max_workers=4):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize or load checkpoint
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
    else:
        checkpoint = {'completed': []}
    
    # Ensure checkpoint is a dictionary with a 'completed' key
    if not isinstance(checkpoint, dict) or 'completed' not in checkpoint:
        checkpoint = {'completed': []}
    
    # Filter out already completed executions
    remaining_executions = [i for i in range(len(parameter_sets)) if i not in checkpoint['completed']]
    
    logging.info(f"Starting execution. {len(remaining_executions)} executions remaining.")
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i in remaining_executions:
            future = executor.submit(
                execute_notebook,
                notebook_to_execute=notebook_to_execute,
                output_dir=output_dir,
                i=i,
                **parameter_sets[i]
            )
            futures.append(future)
        
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                logging.info(f"Execution {result} completed successfully.")
                # Update checkpoint
                checkpoint['completed'].append(result)
                with open(checkpoint_file, 'w') as f:
                    json.dump(checkpoint, f)
            else:
                logging.warning("An execution failed.")
    
    logging.info("All executions completed.")

In [9]:
import random

# Select 4 random elements from parameter_sets
random_parameter_sets = random.sample(parameter_sets, 4)

random_parameter_sets


[{'data_used': 'EM_N_fix_1500',
  'families_to_discard': 2,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'inception_time_wp_vae',
  'latent_dim': 6,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'n_filters': 32,
   'kernel_sizes': [3, 7, 13],
   'bottleneck_channels': 32,
   'beta': 0.5}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard': 0,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'inception_time_wp_vae',
  'latent_dim': 2,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'n_filters': 32,
   'kernel_sizes': [3, 7, 13],
   'bottleneck_channels': 32,
   'beta': 10}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard'

In [ ]:
notebook_to_execute = '03_01_generative_discovery.ipynb'
output_dir = "../experiments/03_01_generative_discovery"
checkpoint_file = '../experiments/experiment_checkpoint.json'

process_parameter_sets(
    parameter_sets,
    notebook_to_execute=notebook_to_execute,
    output_dir=output_dir,
    checkpoint_file=checkpoint_file,
    max_workers=1
)

2024-11-12 12:33:25,378 - INFO - Starting execution. 140 executions remaining.
